In [65]:
# Import
from lxml import html
import requests
import json

In [66]:
# Base URL
base_url = "https://www.olympic.org"

f = open("sports.xml")
tree = html.document_fromstring(f.read())
sports_nodes = tree.xpath("//li")

In [67]:
import time
start = time.time()

events = []

# for s in sports_nodes:

# Get main page.
# sport_dir = s.xpath('a')[0].attrib['href']
sport_dir = "/speed-skating"
url = base_url + sport_dir

print(url)

response = requests.get(url)
tree = html.document_fromstring(response.text)


# Get all events.
sections = tree.xpath("//section[starts-with(@class, 'sport-events')]")

# Iterate through events and extract.
for section in sections[0].xpath("div[@class='sport-event-col']"):

    for elements in section.xpath("div[@class='list-row']")[0]:

        for x in elements.xpath("li"):
            event = {
                "sport": sport_dir.replace("/","").replace("-"," ").title(),
                "name": x.xpath('a')[0].attrib['href'].split("/")[-1].replace("-"," ").title(),
                "url": base_url + "/" + x.xpath('a')[0].attrib['href'],
                "gender": "F" if ("women" in x.xpath('a')[0].attrib['href'].lower()) or ("ladies" in x.xpath('a')[0].attrib['href'].lower()) else "M",
            }
            
            print(event["url"])
            
            response = requests.get(event["url"])
            if "event-box" not in response.text:
                print("404 - PAGE NOT FOUND")
                continue
            tree = html.document_fromstring(response.text)
            event["games"] = []
            games = tree.xpath("//section[@class='event-box']")
            
            for game in games:
                # Access the game specific pages and scrape
                games_name = game.xpath("h2")[0].text_content().strip()
                page = game.xpath("h2")[0].xpath('a')[0].attrib['href']
                game_event_url = base_url + "/" + page
                # print(game_event_url)
                
                response = requests.get(game_event_url)
                tree = html.document_fromstring(response.text)

                place_nodes = tree.xpath("//td[@class='col1']")
                athlete_nodes = tree.xpath("//td[@class='col2']")
                result_nodes = tree.xpath("//td[@class='col3']")

                medals = []
                places = []
                names = []
                nats = []
                results = []

                prelims = False
                previous_place = -1
                # Three medals can be awarded
                medals_awarded = 0
                three_awarded = False
                for p in place_nodes:
                    place = p.text_content().strip()
                    
                    if place == "G" or (place == "1." and not three_awarded):
                        places.append(1)
                        medals.append("Gold")
                        previous_place = 1
                        medals_awarded += 1
                        if medals_awarded == 3:
                            three_awarded = True
                    elif place == "S" or (place == "2." and not three_awarded):
                        places.append(2)
                        medals.append("Silver")
                        previous_place = 2
                        medals_awarded += 1
                        if medals_awarded == 3:
                            three_awarded = True
                    elif place == "B" or (place == "3." and not three_awarded):
                        places.append(3)
                        medals.append("Bronze")
                        previous_place = 3
                        medals_awarded += 1
                        if medals_awarded == 3:
                            three_awarded = True
                    else:
                        if place != "" and (int(place.replace(".","")) < previous_place):
                            # Signals prelims results
                            prelims = True
                        if prelims:
                            # Prelims Finish is empty
                            places.append("")
                        else:
                            if place == "":
                                # Finals No Finish is -1
                                places.append(-1)
                            else:
                                places.append(int(place.replace(".","")))
                                previous_place = int(place.replace(".",""))
                        medals.append("NA")
                
                if medals_awarded != 3:
                    print(str(medals_awarded) + " Medals: " + event["sport"] + "/" + event["name"] + "/" + games_name)
                
                for a in athlete_nodes:
                    arr = a.text_content().split()
                    if len(arr) == 0:
                        nats.append("")
                        names.append("")
                    # Check for valid NOC code
                    elif len(arr[len(arr)-1]) == 3:
                        nats.append(arr.pop())
                    else:
                        nats.append("")
                    names.append(" ".join(arr))

                for r in result_nodes:
                    results.append(" ".join(r.text_content().split()))

                zipped = zip(medals, places, results, names, nats)
                event["games"].append({
                    "location": games_name[:-5],
                    "year": int(games_name[-4:]),
                    "results": [{
                        "medal": z[0],
                        "place": z[1],
                        "result": z[2],
                        "name": z[3],
                        "nationality": z[4]
                    } for z in zipped]
                })
            events.append(event)
end = time.time()
print((end - start)/60)

https://www.olympic.org/speed-skating
https://www.olympic.org//speed-skating/mens-10000m
https://www.olympic.org//speed-skating/mens-1000m
4 Medals: Speed Skating/Mens 1000M/Lake Placid 1980
https://www.olympic.org//speed-skating/mens-1500m
https://www.olympic.org//speed-skating/mens-5000m
https://www.olympic.org//speed-skating/mens-500m
https://www.olympic.org//speed-skating/mens-mass-start
https://www.olympic.org//speed-skating/mens-team-pursuit
2 Medals: Speed Skating/Mens Team Pursuit/Turin 2006
https://www.olympic.org//speed-skating/ladies-1000m
https://www.olympic.org//speed-skating/ladies-1500m
https://www.olympic.org//speed-skating/ladies-3000m
https://www.olympic.org//speed-skating/ladies-5000m
https://www.olympic.org//speed-skating/ladies-500m
https://www.olympic.org//speed-skating/ladies-mass-start
https://www.olympic.org//speed-skating/ladies-team-pursuit
1 Medals: Speed Skating/Ladies Team Pursuit/Turin 2006
3.2939865469932554


In [68]:
# Write JSON
fname = "results_speed-skating"
f = open("./json/" + fname + ".json", "w")
f.write(json.dumps(events, indent=4, sort_keys=True))
f.close()

In [69]:
import codecs

# Write CSV
f = codecs.open("./txt/" + fname + ".txt", "w", "utf-8")
f.write("Sex\tSport\tEvent\tCity\tYear\tMedal\tPlace\tName\tNOC\tResult\n")

for event in events:

    for game in event["games"]:

        for result in game["results"]:
            
            f.write( "\t".join([event["gender"], event["sport"], event["name"], game["location"], str(game["year"]), result["medal"], str(result["place"]), result["name"], result["nationality"], str(result["result"])]) + "\n")
f.close()